In [ ]:
# Import python packages
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


First, download the full dataset (1 million rows) as a zip from this [Kaggle link](https://www.kaggle.com/datasets/sridharstreaks/insurance-data-for-machine-learning). Then unzip it to a .csv. Load that .csv into a table and name it INSURANCE_FULL_DATASET. We will pull from that table to create a training data table and our incoming "streamed" data table.

In [ ]:
# Read the entire table into a Snowpark DataFrame
snowpark_df = session.table('INSURANCE.ML_PIPE.INSURANCE_FULL_DATASET')

# Convert the Snowpark DF to a Pandas DataFrame
insurance_df = snowpark_df.to_pandas()

In [ ]:
# Capitalize column names
insurance_df.columns = insurance_df.columns.str.upper()

# Rearrange columns to fit target schema
cols = insurance_df.columns.tolist()
cols = cols[:3] + cols[-1:] + cols[3:-1]
insurance_df = insurance_df[cols]

In [ ]:
# Write the 10k records to the 'SOURCE_OF_TRUTH' table
source_of_truth_df = session.write_pandas(insurance_df[:10000], table_name='SOURCE_OF_TRUTH',database='INSURANCE',schema='ML_PIPE',auto_create_table=True)

In [ ]:
# Write the remaining 990k records to the 'INCOMING_DATA_SOURCE' table
incoming_data_source_df = session.write_pandas(insurance_df[10000:], table_name='INCOMING_DATA_SOURCE',database='INSURANCE',schema='ML_PIPE',auto_create_table=True)